In [ ]:
%pip install numpy
%pip install pandas
%pip install scikit-learn   
%pip install matplotlib
%pip install tensorflow
%pip install langchain
%pip install langchain-community
%pip install pinecone-client
%pip install -qU langchain-pinecone pinecone-notebooks
%pip install -qU langchain-openai
%pip install -qU langchain-text-splitters
%pip install glob2
%pip install unstructured
%pip install nltk
%pip install ipython-secrets

In [ ]:
!git clone https://github.com/kiritbasu/Fake-Apache-Log-Generator

In [3]:
import glob
import nltk
import unstructured
import pandas as pd
import numpy as np
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

127.0.0.1 - Scott [10/Dec/2019:13:55:36 -0700] "GET /server-status HTTP/1.1" 200 2326

Yukarıdaki log line apache common log formatına örnektir, bu formatta logları yukarıda çektiğim Fake-Apache-Log-Generator reposundan generateleyip üzerinde çalışacağım

    127.0.0.1 - IP address of the client that made the request;
    The hyphen defining the second field in the log file is the identity of the client. This field is often returned as a hyphen and Apache’s HTTP server documentation recommends that this particular field not be relied upon except in the case of a controlled internal network.
    Scott - userid of the person requesting the resource;
    [10/Dec/2019:13:55:36 -0700] - date and time of the request;
    “GET /server-status HTTP/1.1" - request type and resource being requested;
    200 - HTTP response status code;
    2326 - size of the object returned to the client.



In [4]:
loader = DirectoryLoader("Fake-Apache-Log-Generator", glob = "*.log")

In [5]:
logs = loader.load()

In [ ]:
logs[0]

Vectordb olarak pinecone ve llm olarak da chatgpt 3.5 turbo kullandığım için bu notebookun çalışabilmesi için ikisi için de bir api key gerektirmekte.

In [7]:
from ipython_secrets import *
import openai
import os
PINECONE_API_KEY = get_secret("PINECONE_API_KEY")
OPENAI_API_KEY = get_secret("OPENAI_API_KEY")
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

In [8]:
embeddings = OpenAIEmbeddings(
    model = "text-embedding-3-small"
)
index_name = "pineconerag"

In [9]:
text_splitter = RecursiveCharacterTextSplitter()
split_docs = text_splitter.split_documents(logs)

In [ ]:
split_docs[25]

In [57]:
vectorStore = PineconeVectorStore.from_documents(split_docs, embeddings, index_name = index_name)

Bütün sorguları aynı string değişkeniyle atıyorum: query

In [58]:
query = "Whats the IP adress that posted a PUT command in 04/Sep/2024:03:08:37"
similar_docs = vectorStore.similarity_search(query)

In [ ]:
similar_docs[0]

Geri döndürülen dosyada 04/Sep/2024:03:08:37 tarihinde PUT requesti atıldığını görebiliyoruz

In [60]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain

llm = ChatOpenAI(model="gpt-3.5-turbo",temperature = 0)
qa_chain = load_qa_chain(llm, chain_type="stuff")

qa = RetrievalQA(combine_documents_chain=qa_chain, retriever=vectorStore.as_retriever())

In [61]:
response = qa.run(query)

Görüldüğü üzere de llmimiz (chat 3.5 turbo) sistemdeki herhangi bir dosyaya ulaşamadığı için default bir cevap veriyor